In [1]:
from time import sleep

In [2]:
# !wget https://object.pouta.csc.fi/OPUS-UNPC/v1.0/tmx/ar-en.tmx.gz


--2024-05-15 16:22:00--  https://object.pouta.csc.fi/OPUS-UNPC/v1.0/tmx/ar-en.tmx.gz
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.18, 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2093001078 (1.9G) [application/gzip]
Saving to: ‘ar-en.tmx.gz’

ar-en.tmx.gz        100%[===================>]   1.95G  18.8MB/s    in 1m 43s  

2024-05-15 16:23:43 (19.5 MB/s) - ‘ar-en.tmx.gz’ saved [2093001078/2093001078]



In [3]:
sleep(2)

In [4]:
# !gunzip ar-en.tmx.gz


In [5]:
import os
import csv
import math
from lxml import etree

# Constants
TMX_FILE_PATH = './ar-en.tmx'
OUTPUT_DIR = 'UN_CORPUS'
PREFIX = 'bi_ar_en_'
NUM_PARTS = 10

# Ensure output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Function to stream bilingual pairs using iterparse
def stream_bilingual_pairs(file_path):
    context = etree.iterparse(file_path, events=("end",), tag="tu", huge_tree=True)
    for event, elem in context:
        segs = elem.findall("tuv/seg")
        if len(segs) == 2:
            lang1 = elem.findall("tuv")[0].attrib['{http://www.w3.org/XML/1998/namespace}lang']
            lang2 = elem.findall("tuv")[1].attrib['{http://www.w3.org/XML/1998/namespace}lang']
            text1 = segs[0].text
            text2 = segs[1].text
            if lang1 == 'ar' and lang2 == 'en':
                yield (text1, text2)
            elif lang1 == 'en' and lang2 == 'ar':
                yield (text2, text1)
        # Clear the element to free memory
        elem.clear()
        while elem.getprevious() is not None:
            del elem.getparent()[0]
    del context

# Function to write pairs to CSV
def write_to_csv(file_path, pairs):
    with open(file_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Arabic', 'English'])
        writer.writerows(pairs)

# Initialize variables to count pairs and distribute them across files
pairs = []
total_pairs = 0

# Stream pairs and count them
for pair in stream_bilingual_pairs(TMX_FILE_PATH):
    pairs.append(pair)
    total_pairs += 1

# Determine the number of pairs per partition
pairs_per_partition = math.ceil(total_pairs / NUM_PARTS)

# Write partitions to CSV files
for i in range(NUM_PARTS):
    start = i * pairs_per_partition
    end = min((i + 1) * pairs_per_partition, total_pairs)
    csv_file_path = os.path.join(OUTPUT_DIR, f'{PREFIX}{i + 1}.csv')
    write_to_csv(csv_file_path, pairs[start:end])

print('Conversion completed.')

Conversion completed.


In [9]:
import pandas as pd

In [11]:
pd.read_csv('./UN_CORPUS/bi_ar_en_10.csv', encoding='utf-8')

,Arabic,English
0,:: ننادي بأن تُمنح صراحةً على شبكة الإنترنت نف...,:: We advocate that the same rights of freedom...
1,:: نحث إخواننا الشباب على أن يتصرفوا ويقدموا أ...,:: We urge fellow young people to act and pres...
2,:: ندعو إلى وضع سياسات وأطر ملائمة وفعّالة تحم...,:: We call for policies and appropriate and ef...
3,وللتمتع بصحة جيدة:,To be healthy:
4,:: نسعى إلى إقامة أنظمة مبتكرة تسمح بحصول النا...,:: We seek innovative systems which connect pe...
...,...,...
1663677,:: تقرير اللجنة الاستشارية لشؤون الإدارة والمي...,:: Related report of the Advisory Committee on...
1663678,:: جلسة إحاطة حول استراتيجية تعبئة الموارد تكو...,:: Strategic brief on resource mobilization as...
1663679,:: التحليل التجميعي للتقييمات التي أجرتها هيئة...,:: Meta-analysis of evaluations managed by UN-...
1663680,:: التقييم المؤسسي لمساهمة هيئة الأمم المتحدة ...,:: Corporate evaluation of the contribution of...


In [ ]:
# gcloud auth login
# to authenticate, and upload to gcp bucket using gsutil
# gsutil cp -r ./folder gs://bucket-name/folder